In [617]:
from string import punctuation
from stemming.porter2 import stem
from itertools import groupby
import numpy as np

PATH = 'data'
BOOK_NAME = 'book'
FILE_EXTENSION = "txt"

NUMBER_OF_CHAPTERS = 37

#### Divide book by chapters

In [618]:
with open(PATH+'/'+BOOK_NAME+'.'+FILE_EXTENSION, encoding="UTF-8") as f:
    i = 0
    new_chapter = False
    fx = open(PATH+'/'+BOOK_NAME+"_00."+FILE_EXTENSION, 'w')
    for line in f:
        if "CHAPTER" in line:
            i += 1
            new_chapter = True
        if new_chapter:
           new_chapter = False 
           fx.close()
           if i < 10:
            z = "0" + str(i)
           else:
            z = str(i) 
           fx = open(PATH+'/'+BOOK_NAME+'_'+z+'.'+FILE_EXTENSION, 'w') 
        fx.write(line)
    fx.close()    

#### Stopwords

In [619]:
with open(PATH+'/stopwords.'+FILE_EXTENSION, encoding="UTF-8") as f:
    stopwords = [word for line in f for word in line.split()]
    stem_stopwords = [stem(word) for line in f for word in line.split()]

#### Exercise 5

In [620]:
with open(PATH+'/'+BOOK_NAME+'.'+FILE_EXTENSION, encoding="UTF-8") as f:
    words = [word for line in f for word in line.split()]

In [621]:
words = [word.lower().translate(str.maketrans('', '', punctuation)) for word in words]

In [622]:
filtered_words = [w for w in words if not stem(w) in stopwords and not stem(w) in stem_stopwords and not w in stopwords and not w in stem_stopwords and len(w) > 0]

fx = open("results/filtered_words.txt", 'w') 
for fw in filtered_words:
    fx.write(fw + "\n")
fx.close()    

In [623]:
pairs = [(w, 1) for w in filtered_words]

In [624]:
pairs.sort()
word = lambda pair : pair[0]
grouped_pairs = [(w, sum(1 for _ in g)) for w, g in groupby(pairs, key=word)]

In [625]:
occurrences = lambda pair: pair[1]
grouped_pairs.sort(key=occurrences, reverse=True)

In [626]:
fx = open("results/ex5.csv", 'w')
i = 0
dictionary =  {}
for gp in grouped_pairs:
    dictionary.update({gp[0]: [gp[1]] + [0 for _ in range(1, NUMBER_OF_CHAPTERS + 1)]})
    i += 1
    if i > 15:
        fx.write(str(gp[1]) + ',' + gp[0] + "\n")
fx.close() 

![alt text](results/ex5.jpg "cloud")

### Exercise 6

In [627]:
def getGroupedPairs(document):
    with open(document, encoding="UTF-8") as f:
        _words = [word.lower().translate(str.maketrans('', '', punctuation)) for line in f for word in line.split()]
    _filtered_words = [w for w in _words if not stem(w) in stopwords and not stem(w) in stem_stopwords and not w in stopwords and not w in stem_stopwords and len(w) > 0]
    _pairs = [(w, 1) for w in _filtered_words]
    _pairs.sort()
    _word = lambda _pair : _pair[0]
    _grouped_pairs = [(w, sum(1 for _ in g)) for w, g in groupby(_pairs, key=_word)]
    _occurrences = lambda _pair: _pair[1]
    _grouped_pairs.sort(key=_occurrences, reverse=True)
    return _grouped_pairs


In [628]:
for i in range(1, NUMBER_OF_CHAPTERS + 1):
    z = str(i)
    if i < 10:
        z = '0' + str(i)
    chapter = PATH + '/' + BOOK_NAME + '_' + z + '.' + FILE_EXTENSION 
    _grouped_pairs_ = getGroupedPairs(chapter)
    for gp in _grouped_pairs_:
        _tmp = dictionary[gp[0]]
        _tmp[i] = gp[1]
        dictionary.update({gp[0]: _tmp})      
        

In [629]:
def tf(t, d = 0):
    return dictionary[t][d]

def idf(t, D = NUMBER_OF_CHAPTERS):
    f = np.max([np.count_nonzero(dictionary[t])-1, 0])
    df = D / (1 + f)
    idf = np.log(df)
    return np.max([idf, 0])

def tf_idf(t, d, D = NUMBER_OF_CHAPTERS):
    return tf(t, d) * idf(t, D)

In [630]:
files = [open("results/ex6_"+str(i)+".csv", 'w') for i in range(1, NUMBER_OF_CHAPTERS + 1)]

for word in dictionary.keys():
    for i in range(0, NUMBER_OF_CHAPTERS):
        _tfidf = tf_idf(word, i+1)
        if _tfidf > 0:
            files[i].write(str(int(np.floor(_tfidf * 10000))) + ',' + word + "\n")

for file in files:
    file.close()            

![alt text](results/ex6_1.jpg "cloud")
![alt text](results/ex6_2.jpg "cloud")
![alt text](results/ex6_5.jpg "cloud")
![alt text](results/ex6_20.jpg "cloud")

#### Exercise 7

In [631]:
def ex7(word):
    result = [(tf_idf(word, i), i) for i in range(1, NUMBER_OF_CHAPTERS + 1)]
    result.sort(reverse=True)
    result = [r[1] for r in result if r[0] > 0]
    return result
ex7("harry")      

[]

#### Exercise 8

In [632]:
with open(PATH+'/'+BOOK_NAME+'.'+FILE_EXTENSION, encoding="UTF-8") as f:
    _words = [word.lower().translate(str.maketrans('', '', punctuation)) for line in f for word in line.split()]
fw = [w for w in _words if not stem(w) in stopwords and not stem(w) in stem_stopwords and not w in stopwords and not w in stem_stopwords and len(w) > 0]
pairs = [(elem, next_elem) for elem,next_elem in zip(fw, fw[1:]+[fw[0]])]
pairs.sort()
gp = [(w, sum(1 for _ in g)) for w, g in groupby(pairs)]
first_word = lambda gp: gp[0][0]
gp = [(w, [(gg[0][1], gg[1]) for gg in g]) for w, g in groupby(gp, key=first_word)]
# print(gp)
sortkey = lambda gp: gp[1]
gp = [(g[0], sorted(g[1], key=sortkey, reverse=True)[:5]) for g in gp]

dct = { x[0]: x[1] for x in gp }

In [648]:
import random
def predictNextWord(word, _p = True):
    if _p:
        s = sum(p[1] for p in dct[word])
        r = random.randint(1, s)
        for p in dct[word]:
            if r <= p[1]:
                return p[0]
            else:
                r -= p[1]
    else:
        s = sum(1 for p in dct[word])
        r = random.randint(1, s)
        for p in dct[word]:
            if r <= 1:
                return p[0]
            else:
                r -= 1           
    return ""                
def predictSentence(_word, first = False, p = True):
    sentenceLength = random.randint(5, 11)
    word = predictNextWord(_word, p)
    sentence = word.capitalize()
    if first:
        sentence = _word.capitalize() + ' ' + word 
    for _ in range(sentenceLength):
        w = predictNextWord(word, p)
        sentence = sentence + ' ' + w
        word = w
    sentence =  sentence + '.'
    return sentence, word
def predictParagraph(word, p=True):
    paragraphLength = random.randint(5, 12)
    paragraph, word = predictSentence(word, True, p)
    for _ in range(paragraphLength):
        s, w = predictSentence(word, False, p)
        paragraph = paragraph + ' ' + s
        word = w
    return paragraph


In [658]:
predictParagraph("wand", False)

'Wand pointed harry hermione turned ron harry. Ron muttered ron staring crouch bagman brightly. Blue robes pulled open dumbledore calmly checking map er yeah bagman brightly. Blue eyes darting forward began eat bite flobberworm. Draco greatly brow furrowed frank stared. Dumbledore harry harry hermione sat table ron ron harry felt. Ground harry harry ron muttered distractedly releasing. Weasley bill standing harry hermione turned face harry potter. Moody dumbledore bent kissed hem voldemorts father mother father escaped azkaban avery. Voldemort raised hand harry felt wand harry. Felt ground beneath caught sight harry.'